In [12]:
# Setting seeds to try and ensure we have the same results - this is not guaranteed across PyTorch releases.
import torch
import sys
import time
import control
import os
from torch.optim import Adam
torch.manual_seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

import matplotlib.pyplot as plt

import numpy as np
from model_archs import LSTM_FF
from model_archs import LSTM_FF_dropout
import scipy.io
np.random.seed(0)

import h5py
from torch.utils.data import TensorDataset, DataLoader
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

import math
from sklearn.metrics import r2_score

proces_params = {} 

# PARAMETRIZATION
sampl_time = 15/1440 #as in MATLAB files
learning_rate = 1e-3
use_prev_prediction = False
batch_size = 512 
if use_prev_prediction:
    test_batch_size = 1
else:
    test_batch_size = batch_size
num_epochs = 100
test_size = 0.15
early_stop_thresh = 30
num_folds = 5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
train_loader_shuffle = False
kfold_shuffle = True
prev2 = False
break_first_fold = False #if stopping after first fold training
nodrop = False # dropout in model yes or no
custom_loss = False

print(device)

setpoint_signal = './ETROTLE_Dataset_ETFA24_MAT_files/reference/' 
first_order_processes_mat = './ETROTLE_Dataset_ETFA24_MAT_files/PID_tuning_params_1stO_processes.mat'
pids_mat_dir = './ETROTLE_Dataset_ETFA24_MAT_files/1st_order/' 


cpu


In [13]:
def get_inputs_output (process, process_folder):
    """
    Get training data from the .mat files of a certain tuned PID for a certain process.
    Generate input and output vectors.
    """
    mesura_process = f'mesura_PID_{process}'
    actuacio_process = f'actuacio_PID_{process}'
    
    print(mesura_process)
    
    P1_fold = setpoint_signal 

    print(P1_fold)
    
    So5sim = np.array(scipy.io.loadmat(P1_fold + 'referencia_PID_P1.mat')['referencia_PID_P1']).reshape((-1, 1))
    So5out = np.array(scipy.io.loadmat(process_folder + mesura_process +'.mat')[mesura_process]).reshape((-1, 1))
    kla = np.array(scipy.io.loadmat(process_folder + actuacio_process + '.mat')[actuacio_process]).reshape((-1, 1))
    
    error = So5sim - So5out

    # The previously predicted value as an extra input of the model (NARX models in ANNs)
    kla_delayed = np.vstack([kla[0, :], kla[0:len(kla)-1, :]])

    inputs = np.hstack((So5sim, So5out, error, kla_delayed))
    output = kla.reshape((-1,1))

    if prev2:
        kla_delayed2 = np.vstack([kla[0, :], kla[1,:], kla[0:len(kla)-2, :]])
        inputs = np.hstack((So5sim, So5out, error, kla_delayed, kla_delayed2))        
    
    return inputs, output

def get_datasets_dataloaders (inputs, output, bs=batch_size, testbs=test_batch_size):
    # Split the data into training and testing sets
    train_inputs, test_inputs, train_output, test_output = train_test_split(inputs, output, test_size=test_size, random_state=42)

    # Standardize the data separately for training and testing, use scaler for training for scaling test data
    scaler = StandardScaler()
    train_inputs_standardized = scaler.fit_transform(train_inputs)
    test_inputs_standardized = scaler.transform(test_inputs)

    #SCALE OUTPUT DATA

    scaler_out = StandardScaler()
    train_outputs_standardized = scaler_out.fit_transform(train_output)
    test_outputs_standardized = scaler_out.transform(test_output)

    # Convert to PyTorch tensors
    train_inputs_tensor = torch.tensor(train_inputs_standardized, dtype=torch.float32)
    train_output_tensor = torch.tensor(train_outputs_standardized, dtype=torch.float32)

    test_inputs_tensor = torch.tensor(test_inputs_standardized, dtype=torch.float32)
    test_output_tensor = torch.tensor(test_outputs_standardized, dtype=torch.float32)

    # Create PyTorch datasets
    train_dataset = TensorDataset(train_inputs_tensor, train_output_tensor)
    test_dataset = TensorDataset(test_inputs_tensor, test_output_tensor)

    # Create PyTorch DataLoaders
    train_dataloader = DataLoader(train_dataset, batch_size=bs, shuffle=train_loader_shuffle)
    test_dataloader = DataLoader(test_dataset, batch_size=testbs, shuffle=False)
    
    print("Scaler data")
    print(scaler.mean_)
    print(scaler.scale_)

    for batch, (inputs, outputs) in enumerate(train_dataloader, 1):
        print(batch, inputs.size(), outputs.size())
        break

    return train_dataset, test_dataset, train_dataloader, test_dataloader, scaler, scaler_out

def plot_and_save_results(tr_losses, te_losses, suffix, experiment_num = 1):
    plt.figure(figsize=(10, 3))

    plt.subplot(1, 2, 1)
    plt.plot(tr_losses, label='Training Loss', color='blue')
    plt.plot(te_losses, label='Testing Loss', color='red')
    plt.title(f'Training and Testing Loss Over Epochs - Experiment {suffix}')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()  
    plt.tight_layout()
    
    # Show the plot
    file_path = f'Results/Plots/Loss_plot_{process}_{suffix}.png'
    # Save the plot
    plt.savefig(file_path)
    
    plt.show()

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

def mean_absolute_error(output, target):
    return torch.mean(torch.abs(output - target))

def mean_absolute_percentage_error(output, target):
    return torch.mean(torch.abs((target - output) / target)) * 100

def checkpoint(model, filename):
    torch.save(model.state_dict(), filename)
    
def resume(model, filename):
    model.load_state_dict(torch.load(filename))
    
def init_weights(m):
    if isinstance(m, torch.nn.LSTM):
        m.reset_parameters()
    if isinstance(m, torch.nn.Linear):
        m.reset_parameters()    
        
def get_mesura_proces_pidonline(denorm_actuacio, num1, den1, den2, den3, delay, sampling_time = sampl_time, x0 = None):
    """
    Simulate and return process output based on previous state and process dynamics.
    """
    y_out_list = []

    numerator = [num1]
    denominator = [den1, den2, den3]

    # Create the transfer function
    transfer_function = control.TransferFunction(numerator, denominator)

    #PADÉ
    Tau = delay
    N = 10
    [num_pade, den_pade] = control.pade(Tau, N )
    Hpade = control.TransferFunction(num_pade, den_pade)

    # Apply the time delay
    system_with_delay = control.series(transfer_function, Hpade)

    #CONVERT TO STATE SPACE
    ss = control.tf2ss(system_with_delay)

    sample_time = sampling_time
    batch_size = len(denorm_actuacio)

    # Define the time vector
    t = np.linspace(0, (batch_size - 1) * sample_time, batch_size)         
    
    if x0 is None:
        x0 = [0 for i in range(len(ss.A))]
        
    u = denorm_actuacio
    t_out, y_out, x_out = control.forced_response(ss, T=t, U=u, X0=x0, return_x=True)
    #print(u,t_out,y_out)
    #print("X0", x0)
    #print("xout", x_out)
    return y_out.tolist(), t_out, x_out[:,-1] #LIST OF MEASURES OF THE BATCH, LIST OF X0 OF BATCH INSTANCES

        
def IAE_calculator (inputs_iae, model, scaler, scaler_out, pp = "A"):
    """
    One by one simulation pipeline with network recursive predictions over the reference signal,
    365 time, 35041 instances. An instance every 0,010416369 seconds.
    Return control actuations of the model + errors against ground truth.
    """
    actuacions = []
    errors = []

    model.to("cpu")
    model.eval()

    setpoint = inputs_iae[:,0]
    inputs_tensor = torch.empty(inputs_iae[0].shape)
    inputs_tensor = inputs_tensor.unsqueeze(0).unsqueeze(0) #1,1,4 shape
    counter = 1
    X0 = None
    hidden = None

    if prev2:
        prediction = inputs_iae[0,4]
    
    for ref in setpoint: 
        if counter == 1:
            inputs_0_norm = (inputs_iae[0] - scaler.mean_) / scaler.scale_
            inputs_tensor = torch.tensor(inputs_0_norm ,dtype=torch.float32).to("cpu")
            inputs_tensor = inputs_tensor.unsqueeze(0).unsqueeze(0)

            denorm_actuacio_previa = np.array([inputs_iae[0,3]])[0]
            
        else:
            ref_norm = (ref - scaler.mean_[0]) / scaler.scale_[0]
            inputs_tensor[:,:,0] = ref_norm
            inputs_tensor[:,:,3] = prediction
            #NEW
            if prev2:
                inputs_tensor[:,:,4] = prev2_prediction
            denorm_actuacio = scaler_out.scale_ * output_prev_ann + scaler_out.mean_
            actuacions.append(denorm_actuacio)

            denorm_actuacio = [denorm_actuacio_previa, denorm_actuacio[0]]
            mesura_lstm, _ , X0 = get_mesura_proces_pidonline(denorm_actuacio, pp[0], pp[1], pp[2], pp[3], pp[4], x0 = X0)
            denorm_actuacio_previa = denorm_actuacio[-1]
            mesura_lstm = torch.tensor(mesura_lstm[-1]) 

            #Normalize mesura_lstm for putting in inputs
            mesura_lstm_norm = (mesura_lstm - scaler.mean_[1]) / scaler.scale_[1]
            inputs_tensor[:,:,1] = mesura_lstm_norm
            
            #error is denorm ref-mesura and then norm
            error = ref - mesura_lstm
            errors.append(ref - mesura_lstm.item())
            inputs_tensor[:,:,2] = (error - scaler.mean_[2]) / scaler.scale_[2]
            
        with torch.no_grad():
            if prev2:
                prev2_prediction = prediction
            
            prediction = model(inputs_tensor)
            predictions_list = [elem[0] for elem in prediction]

            predictions = torch.tensor(predictions_list)
            output_prev_ann = predictions.numpy().tolist()
        counter +=1
        
        #print(counter, inputs_tensor, prediction)
    return actuacions, errors  
    
class ClassicMSELoss(torch.nn.Module):
    def __init__(self):
        super(ClassicMSELoss, self).__init__()

    def forward(self, predictions, targets, error, batch_size):
        # Compute the standard MSE loss
        loss = torch.mean((predictions - targets) ** 2)
        return loss


class CustomMSELoss(torch.nn.Module):
    """
    Created for testing purposes, not used in practicality.
    """
    def __init__(self):
        super(CustomMSELoss, self).__init__()
        self.iterations = 0
        self.total_integral_error = 0.0

    def forward(self, predictions, targets, error, batch_size):
        # Compute the standard MSE loss
        mse_loss = torch.mean((predictions - targets) ** 2)
        
        # Compute the integral of the error and accumulate it
        self.total_integral_error += torch.sum(error)
        
        # Update the number of iterations
        self.iterations += batch_size
        
        # Calculate the integral error term
        #integral_error_term = self.total_integral_error / self.iterations
        integral_error_term = error
        #print(self.iterations, self.total_integral_error, integral_error_term)

        # Weight for the integral error term
        weight = 10  # Adjust this parameter as desired
        
        # Add the integral error term to the MSE loss
        loss = mse_loss + weight * integral_error_term
        
        return loss


In [14]:
def train_model (model, train_dataset, transfer=False, hdf5_paths=""):
    """
    Traditional batch-epoch based training of the network with the train_dataset
    data of the PID behavior. Early stopping inside folds and checkpointing of 
    best model of the Kfold training. Return model and loss signals.
    
    transfer=True when starting point from an existing model on path = hdf5_paths.
    """
    print(model.to(device))

    #optimizer = Adam(model.parameters(), lr=learning_rate, weight_decay=0.001)
    optimizer = Adam(model.parameters(), lr=learning_rate)

    if custom_loss:
        criterion = CustomMSELoss()
    else:
        criterion = ClassicMSELoss()  
    print(f'criterion is {str(criterion)}')

    kf = KFold(n_splits=num_folds, shuffle=kfold_shuffle)
    
    best_val = sys.maxsize

    start_time = time.time()
    fold_val_loss = []
    
    tr_loss = []
    te_loss = []
    
    # Loop over folds
    for fold, (train_idx, val_idx) in enumerate(kf.split(train_dataset)):
        print(f"\nFold {fold + 1}/{num_folds}")

        if(transfer == False):
            print("Reset model parameters no transfer")
            model.apply(init_weights)

        else:
            print("Transfer true, load weights")
            model = load_model_from_h5(model, hdf5_paths)
            model.to(device)

        # Create dataloaders for this fold
        train_fold_dataset = torch.utils.data.Subset(train_dataset, train_idx)
        val_fold_dataset = torch.utils.data.Subset(train_dataset, val_idx)

        train_fold_loader = DataLoader(train_fold_dataset, batch_size=batch_size, shuffle=train_loader_shuffle)
        val_fold_loader = DataLoader(val_fold_dataset, batch_size=batch_size, shuffle=False)

        best_val_e = sys.maxsize
        best_epoch = 0

        # Training loop for each fold
        for e in range(num_epochs):
            batch_loss = 0
            
            #Reboot loss per epoch
            if custom_loss:
                criterion = CustomMSELoss()
            else:
                criterion = ClassicMSELoss()  

            # Training phase
            model.train()
            for batch, (inputs, outputs) in enumerate(train_fold_loader, 1):
                inputs = inputs.to(device).unsqueeze(1)
                outputs = outputs.to(device)
                optimizer.zero_grad()
                predictions = model(inputs)
                loss = criterion(predictions, outputs, inputs[:,0,2], batch_size)
                loss.backward()
                optimizer.step()

                batch_loss += loss.item()
                #print(f'Epoch({e + 1}/{num_epochs}: Batch number({batch}/{len(train_fold_loader)}) : Batch loss : {loss.item()}')
            print(f'Epoch({e + 1}/{num_epochs})')
            print(f'Training loss : {batch_loss / len(train_fold_loader)}')
            tr_loss.append(batch_loss / len(train_fold_loader))
            
            criterion = torch.nn.MSELoss() #default choice
            
            # Validation phase
            model.eval()
            val_loss = 0
            with torch.no_grad():
                for batch_val, (inputs, outputs) in enumerate(val_fold_loader, 1):
                    inputs = inputs.to(device).unsqueeze(1)
                    outputs = outputs.to(device)
                    predictions = model(inputs)
                    loss = criterion(predictions, outputs)
                    
                    val_loss += loss.item()
                print(f'Validation loss : {val_loss / len(val_fold_loader)}')
                te_loss.append(val_loss / len(val_fold_loader))

                val_epoch = val_loss / len(val_fold_loader)

                #Best model checkpointing
                if(val_epoch < best_val):
                    print(f'Checkpointing new best model in epoch {e+1} with fold {fold+1}')
                    checkpoint(model, f"./Temp_Models/best_model.pkl")
                    best_val = val_epoch

            #Check val loss of epoch for early stopping
            if val_epoch < best_val_e:
                best_val_e = val_epoch
                best_epoch = e

            #Early stopping
            if e - best_epoch > early_stop_thresh:
                print(f'Early stopped training at epoch {e+1}')
                break  # terminate the training loop

        #At the end of all the epochs
        print(f'Checkpointing epoch {e+1} and fold {fold+1} ')
        checkpoint(model, f"./Temp_Models/fold-{fold+1}.pkl")

        fold_val_loss.append(best_val_e)
        
        if break_first_fold:
            print("BREAKING FIRST FOLD")
            break
        
    print(f'\nFinal, best val error: {best_val}')
    print(f'Avg val loss CV is: {sum(fold_val_loss)/len(fold_val_loss)}')

    end_time = time.time()
    elapsed_time = end_time - start_time
    minutes = int(elapsed_time // 60)
    seconds = int(elapsed_time % 60)

    print(f"Elapsed Time: {minutes} minutes and {seconds} seconds")
    
    return model, tr_loss, te_loss

    
def evaluate_model (model, test_dataloader, use_prediction = False):
    """
    Evaluate model on test set and plot it with metrics, against real 
    PID ground_truth or with previous actuation fed in input.
    """
    
    device = 'cpu'
    model.to(device)
    model.eval()
    test_loss = 0
    predictions_all = []
    outputs_all = []
    criterion = torch.nn.MSELoss()
    prev_prediction = None
    
    print ("Evaluation with real labels PID")

    with torch.no_grad(): #deactivate dropout regularization, if there is
        for batch_val, (inputs, outputs) in enumerate(test_dataloader, 1):
            inputs = inputs.to(device).unsqueeze(1)
            outputs = outputs.to(device)
            
            predictions = model(inputs)

            loss = criterion(predictions, outputs)
            test_loss += loss.item()
            predictions_all.append(predictions.cpu().numpy())
            outputs_all.append(outputs.cpu().numpy())
            
    predictions_all = np.concatenate(predictions_all, axis=0)
    outputs_all = np.concatenate(outputs_all, axis=0)
    r2 = r2_score(outputs_all, predictions_all)
    mape = np.mean(np.abs((outputs_all - predictions_all) / outputs_all)) * 100
    rmse = math.sqrt(test_loss / len(test_dataloader))
    
    print(f'RMSE: {rmse}')
    print(f'R^2: {r2}')
    print(f'MAPE: {mape}%')
    
    # Plot predictions vs. outputs
    plt.figure(figsize=(10, 6))
    plt.plot(outputs_all, 'b-', label='Actual')  # Blue solid line for actual
    plt.plot(predictions_all, 'r--', label='Predicted')  # Red dashed line for predicted
    plt.xlabel('Sample')
    plt.ylabel('Value')
    plt.title('Actual vs Predicted')
    plt.legend()
    plt.grid(True)
    plt.show()


def save_model_as_h5(model, process, suffix):
    state_dict = model.state_dict()

    with h5py.File(f'./Temp_Models/model_weights_{process}_{suffix}.h5', 'w') as f:
        # Save each tensor in the state_dict as a dataset in the HDF5 file
        for key, value in state_dict.items():
            f.create_dataset(key, data=value.numpy())

def load_model_from_h5 (model, path):
    # Load the weights from the HDF5 file
    with h5py.File(path, 'r') as f:
        # Iterate through the layers in your PyTorch model
        for name, param in model.named_parameters():
            # Check if the layer name exists in the HDF5 file
            if name in f:
                # Load the weights from the HDF5 file to the PyTorch model
                param.data = torch.tensor(f[name][:])
    #Whole model trainable
    for param in model.parameters():
        param.requires_grad = True
    
    return model
                       
            
def train_and_save_model (process, process_folder, suffix, third=False, transfer_bool = False,
                          transfer_suffix = "", process_from_transfer = " "):
    """
    Train and save model (from scratch or transferring weights), evaluate it against PID,
    calculate errors, show them and save plots and metrics.
    """
    inputs, output = get_inputs_output(process, process_folder)
            
    train_dataset, test_dataset, train_dataloader, test_dataloader2, scaler, scaler_out = get_datasets_dataloaders (inputs, output)

    if nodrop:
        model = LSTM_FF()
    else:
        model = LSTM_FF_dropout()
 
    #transfer
    hdf5_path = ""
    if transfer_bool:
        hdf5_path = f'Temp_Models/model_weights_{process}_{suffix}.h5'

    #Train model, best model will be in best_model.pkl as well
    model_trained, tr_loss, te_loss = train_model(model, train_dataset, transfer= transfer_bool, hdf5_paths=hdf5_path)

    model_state_dict = torch.load("./Temp_Models/best_model.pkl")
    model.load_state_dict(model_state_dict)
    
    print("suffix before", suffix)
    
    if transfer_bool:
        suffix = suffix + f'_transfer_{transfer_suffix}'
    
    print("suffix after", suffix)
    
    plot_and_save_results(tr_loss, te_loss, suffix)

    #Evaluate over PID ground-truth input-outputs
    evaluate_model(model, test_dataloader2, use_prediction=False)

    print(f'IAE and ISE of model for {process}')
    actuacions, errors = IAE_calculator (inputs, model, scaler, scaler_out, proces_params[process])
    # Flatten each array and concatenate into a single list
    actuacions_list = [number for array in actuacions for number in array.flatten()]
    
    # Calculate the integral absolute error
    IAE = sum(abs(error) for error in errors) * sample_time
    print("Integral Absolute Error (IAE):", IAE)
    # Calculate the integral squared error
    ISE = sum(error**2 for error in errors) * sample_time
    print("Integral Squared Error (ISE):", ISE)
    plt.figure(figsize=(10, 6))  
    plt.plot(output[1:], label='PID', linewidth=1)  
    plt.plot(actuacions_list, label='ANN', linewidth=1) 
    plt.xlabel('Sample')
    plt.ylabel('Control signal')
    plt.legend()
    plt.title('Comparison of PID and ANN control signal')
    # Show plot  
    #NEW
    
    RMSE = np.sqrt(np.mean((np.array(output[1:]) - np.array(actuacions)) ** 2))
    print("Root Mean Squared Error (RMSE):", RMSE)
        
    # Define the file path to save the plot
    file_path = f'./Results/Plots/IAE_ISE_plot_{process}_{suffix}.png'
    # Save the plot
    plt.savefig(file_path, dpi=200)
    
    values_file_path = f'./Results/IAE_ISE_{suffix}.txt'
    
    if not os.path.exists(values_file_path):
        with open(values_file_path, 'w') as f:
            f.write(f'Process: {process}\n')
            # Write IAE and ISE values
            f.write(f'IAE: {IAE}\nISE: {ISE}\n')
            # Write process variable
            f.write(f'RMSE: {RMSE} \n')
            # Write process variable
    else:
        with open(values_file_path, 'a') as f:
            f.write(f'Process: {process}\n')
            # Write IAE and ISE values
            f.write(f'IAE: {IAE}\nISE: {ISE}\n')
            # Write process variable
            f.write(f'RMSE: {RMSE} \n')
    
    
    plt.show()    

    if transfer_bool:
        save_model_as_h5(model, process_from_transfer, suffix)
    
        pkl_path = f'./Temp_Models/model_weights_{process_from_transfer}_{suffix}.pkl'
    # Save the PyTorch model as a .pkl file
        torch.save(model.state_dict(), pkl_path)
    else:
        save_model_as_h5(model, process, suffix)
    
        pkl_path = f'./Temp_Models/model_weights_{process}_{suffix}.pkl'
    # Save the PyTorch model as a .pkl file
        torch.save(model.state_dict(), pkl_path)


### Execution --> train NS models, 100 epochs, loop for 1-10 FOPDT processes

In [16]:
 # Load the .mat file
data = scipy.io.loadmat(first_order_processes_mat)

# Extract parameter values
K_values = data['K_values'][0]
T_values = data['T_values'][0]
alpha_values = data['alpha_values'][0]
Kp_values = data['Kp_values'][0]
Ti_values = data['Ti_values'][0]
Td_values = data['Td_values'][0]
beta_values = data['beta_values'][0]
IAE_values = data['IAE_values'][0]
tao_values = data['tao_values'][0]
L_values = data['L_values'][0]

#Default parametrization
num_epochs = 100
early_stop_thresh = 30
suffix = 'NS_test'
suffix_transfer = ''
nodrop = False
prev2 = False
custom_loss = False
break_first_fold = False

for i in [0,1,2,3,4,5,6,7,8,9]: #the 10 processes
    process = str(i+1)
    print("process", process)
    proces_params[process] = [K_values[i], 0, T_values[i], 1, L_values[i]]
    beta = beta_values[i]
    print("process_params['process']", proces_params[process] )

    P_folder = pids_mat_dir
    train_and_save_model(process, P_folder, suffix, third=False, transfer_bool= False, transfer_suffix= suffix_transfer)

process 1
process_params['process'] [1.4, 0, 1.2, 1, 0.13]
mesura_PID_1
./ETROTLE_Dataset_ETFA24_MAT_files/reference/
Scaler data
[9.91231767e-01 9.90758943e-01 4.72824175e-04 7.09528959e-01]
[0.39851616 0.40047368 0.07421704 0.35155008]
1 torch.Size([512, 4]) torch.Size([512, 1])
LSTM_FF_dropout(
  (lstm1): LSTM(4, 100, batch_first=True)
  (lstm2): LSTM(100, 50, batch_first=True)
  (dropout_lstm): Dropout(p=0.25, inplace=False)
  (fc1): Linear(in_features=50, out_features=25, bias=True)
  (relu1): ReLU()
  (dropout_dense): Dropout(p=0.25, inplace=False)
  (output_layer): Linear(in_features=25, out_features=1, bias=True)
)
criterion is ClassicMSELoss()

Fold 1/5
Reset model parameters no transfer
Epoch(1/100)
Training loss : 0.7932291645952996
Validation loss : 0.3788617129127185
Checkpointing new best model in epoch 1 with fold 1
Epoch(2/100)
Training loss : 0.164494025025596
Validation loss : 0.07237972551956773
Checkpointing new best model in epoch 2 with fold 1
Epoch(3/100)
Trainin

KeyboardInterrupt: 